In [1]:
# Import packages
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix, ConfusionMatrixDisplay, accuracy_score

# Reading the Data

In [2]:
train_data = pd.read_csv("../Data/train_data.csv")
val_data = pd.read_csv("../Data/validation_data.csv")
test_data = pd.read_csv("../Data/test_data.csv")

train_data.sample(5)

,Unnamed: 0,text,class_label,text_preprocessed
22729,5630,Florida GOP’s Anti-Abortion Law Struck Down I...,1,florida gop antiabort law struck anoth huge pl...
9926,896,Trump taps Fed centrist Powell to lead U.S. ce...,0,trump tap fed centrist powel lead us central b...
13822,18759,Coulter Calls on Trump to Impose Temporary Ban...,0,coulter call trump impos temporari ban immigr ...
25693,7166,"San Francisco Cops Caught Sending Racist, Hom...",1,san francisco cop caught send racist homophob ...
12098,348,"Trump’s UN Speech May Have Been Terrible, But...",1,trump un speech may terribl face repres pricel...


In [3]:
X_train_text = train_data["text_preprocessed"].values
y_train = train_data["class_label"].values

X_val_text = val_data["text_preprocessed"].values
y_val = val_data["class_label"].values

X_test_text = test_data["text_preprocessed"].values
y_test = test_data["class_label"].values

# Baseline Model (Random Forest)
## Using CountVectoriser with Bag of Words, Bigrams

In [4]:
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,1))
vectorizer.fit(X_train_text)

X_train = vectorizer.transform(X_train_text)
X_val = vectorizer.transform(X_val_text) 
X_test = vectorizer.transform(X_test_text)

In [5]:
# initialise Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=0) 

# train model
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [6]:
y_val_pred = clf.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.92      0.95      0.93      6361
           1       0.95      0.92      0.94      6659

    accuracy                           0.93     13020
   macro avg       0.93      0.94      0.93     13020
weighted avg       0.94      0.93      0.93     13020



In [7]:
y_test_pred = clf.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94      6361
           1       0.95      0.93      0.94      6660

    accuracy                           0.94     13021
   macro avg       0.94      0.94      0.94     13021
weighted avg       0.94      0.94      0.94     13021



In [8]:
# get predictions and compare results
predictions = clf.predict(X_test)
results = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
results.sample(10)

,Actual,Predicted
6158,0,0
2369,1,1
1683,0,1
4610,0,0
11092,1,1
4177,1,1
10912,0,0
7637,1,1
7443,0,0
7685,1,1


In [9]:
# Accuracy
accuracy = accuracy_score(y_test,predictions)
print("Accuracy:",accuracy)

Accuracy: 0.9398663697104677


In [10]:
# Precision and Recall
precision = metrics.precision_score(y_test, predictions)
recall = metrics.recall_score(y_test, predictions)
f_measure = metrics.f1_score(y_test, predictions)
print("Precision:",precision)
print("Recall:",recall)
print("F-measure:",f_measure)

Precision: 0.9467842481374487
Recall: 0.934984984984985
F-measure: 0.940847624084007


# Using Tf-Idf and Unigrams

In [11]:
tfidf_params = {'unigram':(1,1), 'unigram and bigram': (1,2), 'bigram':(2,2)}

for ngram, values in tfidf_params.items():
    tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=values)
    tfidf_vectorizer.fit(X_train_text)

    X_train = tfidf_vectorizer.transform(X_train_text)
    X_val = tfidf_vectorizer.transform(X_val_text)
    X_test = tfidf_vectorizer.transform(X_test_text)

    print(f"Model with {ngram}")
    clf.fit(X_train, y_train)

    # Validation Data
    print("Testing using validation data:")    
    y_val_pred = clf.predict(X_val)
    print(classification_report(y_val, y_val_pred))
    print("------------------------------------------")

    # Test Data
    print("Testing using test data:")
    y_test_pred = clf.predict(X_test)
    print(classification_report(y_test, y_test_pred))
    print("------------------------------------------")
    print("------------------------------------------")

Model with unigram
Testing using validation data:
              precision    recall  f1-score   support

           0       0.93      0.94      0.93      6361
           1       0.94      0.93      0.94      6659

    accuracy                           0.94     13020
   macro avg       0.93      0.94      0.93     13020
weighted avg       0.94      0.94      0.94     13020

------------------------------------------
Testing using test data:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      6361
           1       0.94      0.94      0.94      6660

    accuracy                           0.94     13021
   macro avg       0.94      0.94      0.94     13021
weighted avg       0.94      0.94      0.94     13021

------------------------------------------
------------------------------------------
Model with unigram and bigram
Testing using validation data:
              precision    recall  f1-score   support

           0       0.93   